In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../kaggle_prediction_library/') 
import preprocess
import feature_engineering
import submission
from sklearn.model_selection import train_test_split

# from hyperopt import tpe, fmin, Trials
# import hyperopt.hp as hp


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss


### Read data

In [ ]:
# For now we can use this, but we may need to recreate this once the teams have been announced
# The new version should only include feasible tourney matchups to make life easier
sub_df = pd.read_csv('../data/SampleSubmission2023.csv')

In [12]:
regular_season_results = pd.read_csv('../data/MRegularSeasonDetailedResults.csv')
detailed_tourney_results = pd.read_csv('../data/MNCAATourneyDetailedResults.csv')
rankings = pd.read_csv('../data/MMasseyOrdinals.csv')
seeds = pd.read_csv('../data/MNCAATourneySeeds.csv')

# kp_rankings = pd.read_csv('../data/kenpom_pre_tourney_snapshot.csv')

regular_season_results_w = pd.read_csv('../data/WRegularSeasonDetailedResults.csv')
detailed_tourney_results_w = pd.read_csv('../data/WNCAATourneyDetailedResults.csv')

mteams = pd.read_csv('../data/MTeams.csv')
wteams = pd.read_csv('../data/WTeams.csv')

seeds_w = pd.read_csv('../data/WNCAATourneySeeds.csv')

# M538 = pd.read_csv('../data/M538.csv')
# W538 = pd.read_csv('../data/W538.csv')

seed_round = pd.read_csv("../data/MNCAATourneySeedRoundSlots.csv")
seeds = pd.read_csv("../data/MNCAATourneySeeds.csv")

first_round_odds_data = pd.read_csv('../data/first_round_odds_data.csv')



In [13]:
# This is temporary, will remove once we are predicting 2024 instead of 2023
detailed_tourney_results = detailed_tourney_results[detailed_tourney_results.Season < 2023].copy()
detailed_tourney_results_w = detailed_tourney_results_w[detailed_tourney_results_w.Season < 2023].copy()

### Set Up Data

In [14]:
to_predict_mens, to_predict_womens, regular_season_games, regular_season_games_w = preprocess.full_setup(detailed_tourney_results, regular_season_results,
               detailed_tourney_results_w, regular_season_results_w,
               sub_df, mteams)

### Add Features

In [15]:
to_predict_womens = feature_engineering.TournamentSeed(tourney_seeds=seeds_w).add(to_predict_womens)
to_predict_womens = feature_engineering.Efficiency(games=regular_season_games_w, away_bonus=0).add(to_predict_womens)
# to_predict_womens = feature_engineering.FiveThirtyEight(fivethirtyeight_df=W538).add(to_predict_womens)

In [16]:
to_predict_mens = feature_engineering.TeamNames(mteams).add(to_predict_mens)
to_predict_mens = feature_engineering.FirstRoundOdds(first_round_odds_data).add(to_predict_mens)

In [17]:
to_predict_mens = feature_engineering.SeasonStats(regular_season_games).add(to_predict_mens)
# to_predict_mens = feature_engineering.FiveThirtyEight(fivethirtyeight_df=M538).add(to_predict_mens)
to_predict_mens = feature_engineering.PreSeasonAPRankings(rankings_df=rankings).add(to_predict_mens)
to_predict_mens = feature_engineering.TournamentSeed(tourney_seeds=seeds).add(to_predict_mens)
to_predict_mens = feature_engineering.Efficiency(games=regular_season_games, away_bonus=3).add(to_predict_mens)
to_predict_mens = feature_engineering.FinalRanking(rankings_df=rankings, system='SAG').add(to_predict_mens)
# to_predict_mens = feature_engineering.Kenpom(kp_snapshot=kp_rankings).add(to_predict_mens)

# this one takes 3 minutes to run
# to_predict_mens = feature_engineering.TeamQuality(games=regular_season_games).add(to_predict_mens)

to_predict_mens = feature_engineering.RoundNumber(seeds, seed_round).add(to_predict_mens)



In [18]:
to_predict_mens.head()

,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,margin,t1_TeamName,...,seed_diff,t1_adj_oe,t1_adj_de,t1_adj_margin,t2_adj_oe,t2_adj_de,t2_adj_margin,t1_final_rank,t2_final_rank,GameRound
0,Historical,2003_1421_1411,None,2003,1421,1411,1,M,8,UNC Asheville,...,0.0,181.059599,66.435419,114.624180,126.485376,74.186910,52.298467,66.473193,66.106343,0.0
1,Historical,2003_1112_1436,None,2003,1112,1436,1,M,29,Arizona,...,-15.0,33.143472,169.541358,-136.397887,130.650668,-13.354714,144.005383,95.514642,72.542565,1.0
2,Historical,2003_1113_1272,None,2003,1113,1272,1,M,13,Arizona St,...,3.0,-38.644484,144.142281,-182.786765,-28.964143,125.192410,-154.156553,84.559424,86.033588,1.0
3,Historical,2003_1141_1166,None,2003,1141,1166,1,M,6,C Michigan,...,5.0,200.421808,149.727556,50.694252,53.177500,122.849582,-69.672082,82.250901,87.404062,1.0
4,Historical,2003_1143_1301,None,2003,1143,1301,1,M,2,California,...,-1.0,46.115283,169.330206,-123.214923,-43.554173,205.005569,-248.559742,83.327530,82.508501,1.0


### Split Dataset

In [19]:
first_round_df = to_predict_mens[to_predict_mens.GameRound == 1].copy()
other_rounds_df = to_predict_mens[to_predict_mens.GameRound > 1].copy()

In [20]:
# other_rounds_df = feature_engineering.FirstRoundOpponentQuality(first_round_df, other_rounds_df).add()


In [21]:
first_round_df.to_csv("../development_notebooks/to_predict_mens_first_round.csv")
other_rounds_df.to_csv("../development_notebooks/to_predict_mens_other_rounds.csv")


In [25]:
to_predict_mens.to_csv("../development_notebooks/to_predict_mens.csv")
to_predict_mens.to_csv("../production_notebooks/to_predict_mens.csv")

In [24]:
other_rounds_df.groupby("type").count()

,ID,Pred,Season,Team1,Team2,Outcome,Gender,margin,t1_TeamName,t1_FirstD1Season,...,seed_diff,t1_adj_oe,t1_adj_de,t1_adj_margin,t2_adj_oe,t2_adj_de,t2_adj_margin,t1_final_rank,t2_final_rank,GameRound
type,,,,,,,,,,,,,,,,,,,,,
Historical,1178,0,1178,1178,1178,1178,1178,1178,1178,1178,...,1178,1178,1178,1178,1178,1178,1178,1178,1178,1178
Prediction,2238,2238,2238,2238,2238,0,2238,0,2238,2238,...,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238
